In [31]:
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_groq import ChatGroq

# Step 1: Load environment variables from .env file
load_dotenv("../.env")

# Step 2: Retrieve Azure OpenAI environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

# Step 3: Initialize the Azure OpenAI model with required parameters
model = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",  # Your Azure deployment
    api_version=OPENAI_API_VERSION,  # Your API version
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,  # Controls randomness; 0 is more deterministic
    max_tokens=60,  # Set according to your needs
    timeout=None  # Adjust if needed
)

model_35 = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  # Your Azure deployment
    api_version=OPENAI_API_VERSION,  # Your API version
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,  # Controls randomness; 0 is more deterministic
    max_tokens=60,  # Set according to your needs
    timeout=None  # Adjust if needed
)

model_llama3 = ChatGroq(
    model="llama-3.3-70b-specdec",
    temperature=0,
    max_tokens=60,
    timeout=None,
    # max_retries=2,
    # other params...
)


import pandas as pd
df = pd.read_csv("random_samples_new_revised.csv")
# print(df.head())

import numpy as np
# Use iloc to select the row by index
samples = df[df["Gold Label"]=="yes"].reset_index() 
random_index = np.random.choice(samples.index)
random_samples = samples.iloc[random_index]
random_samples

# Generate 100 random indices from a range of 0 to 1000 (exclusive)
# random_indices = np.random.randint(0, len(samples), size=100)

# print(random_indices)

# random_samples = samples.iloc[random_indices]
# random_samples

level_0                                                         34
index                                                          393
ID                                                             393
Country                                                     russia
Background       ### Visiting\n- When visiting a Russian home, ...
Axis                                                     Etiquette
Subaxis                                                   visiting
Value                 Hospitality and reciprocity in interactions.
Rule-of-Thumb    It is respectful to bring gifts when visiting ...
Explanation      Bringing a gift when visiting someone's home a...
Gold Label                                                     yes
Story            Emily went to visit Mark's house for the first...
social_action    Emily brought a bottle of wine and some chocol...
new_stories      What would be Emily's appropriate actions as s...
revised_story    What would be Emily's appropriate actions as 

In [32]:
# Step 5: Create a NormBank prompt template for generating dialogue
action_template = """
You are a cultural expert. There is a question to consider:

Story: 
{question}

Action: Provide a brief, culturally-informed response to the question based on {country}. Make it concise.
"""

action_prompt = PromptTemplate(
    template=action_template,
    input_variables=["question", "country"]
)

# Step 6: Define output parser (just as you have done in the reference)
output_parser = StrOutputParser()

# Step 7: Chain the components together (prompt + model + output parser)
action_chain = action_prompt | model_llama3 | output_parser

# Step 8: Define input values and invoke the chain to generate the dialogue
input_values = {
    "question" : random_samples['revised_story'],
    "country" : random_samples['Country']
}

# Step 9: Generate the result
action = action_chain.invoke(input_values)
print("="*20, " Revised Story ", "="*20)
print(random_samples["revised_story"])
print("="*20, " Action ", "="*20)
print(action)
print("="*20, " Explanation ", "="*20)
print(random_samples["Explanation"])


# 348
# ==================================================
# In Germany, as Sarah arrives at the café to meet her friends, an appropriate action would be to greet them with a handshake or a kiss on each cheek (depending on the level of familiarity), and say "Hallo" or "Guten Tag" (hello or good day). It's also customary
# ==================================================
# Given the cultural context, Sarah's actions demonstrate respect for punctuality, personal space, and social etiquette, making her actions socially acceptable.

====================  Revised Story  ====================
What would be Emily's appropriate actions as she visited Mark's house for the first time?
====================  Action  ====================
In Russia, when visiting someone's home for the first time, it's customary to bring a gift, such as flowers, chocolates, or a bottle of wine. Emily should also remove her shoes before entering, as many Russian homes have a no-shoes policy. Additionally, she should greet Mark with
====================  Explanation  ====================
Bringing a gift when visiting someone's home and offering to help clean after meals are seen as respectful gestures. Emily's actions are socially acceptable given the cultural context.


In [34]:
# entail_template = """
# Classify the Textual entailment of given ground truth and action, answer only "entailment" or "contradiction"

# Response Format:
# Social Action : social action of response
# Entailment: entailment/contradiction
# Reasoning: rational information\

# Explanation: {explanation}
# Acton : {action}
# Output:
# """

# entail_prompt = PromptTemplate(
#     template=entail_template,
#     input_variables=["ground_truth", "action"]
# )

# # Step 6: Define output parser (just as you have done in the reference)
# output_parser = StrOutputParser()

# # Step 7: Chain the components together (prompt + model + output parser)
# entail_chain = entail_prompt | model | output_parser

# # Step 8: Define input values and invoke the chain to generate the dialogue
# input_values = {
#     "explanation" : sample['Explanation'],
#     "action" : action
# }

# # Step 9: Generate the result
# text_entailment = entail_chain.invoke(input_values)
# print(text_entailment)

In [35]:
# accept_template = """
# {story}
# Answer only `yes`, `no` or `neutral`
# """

# accept_prompt = PromptTemplate(
#     template=accept_template,
#     input_variables=["story"]
# )

# # Step 6: Define output parser (just as you have done in the reference)
# output_parser = StrOutputParser()

# # Step 7: Chain the components together (prompt + model + output parser)
# accept_chain = accept_prompt | model | output_parser

# # Step 8: Define input values and invoke the chain to generate the dialogue
# input_values = {
#     "story" : sample['Story'],
# }

# # Step 9: Generate the result
# acceptablility = accept_chain.invoke(input_values)
# acceptablility

In [36]:
# print(f""" 
# Story: 
# {sample["Story"]}
# Country: 
# {sample['Country']}
# Label:
# {sample['Gold Label']}
# --------------------------------------------------------
# Question: 
# {questions}
# --------------------------------------------------------
# Explanation:
# {sample['Explanation']}

# Action:
# {action}
# ------------------------------------------------
# NLI :
# {text_entailment}
# Acceptable : {acceptablility}
# """)